# Introduction

## Overview

In the realm of gravitational wave (GW) astrophysics, particularly concerning the statistical analysis of compact binary coalescences (CBC) and hierarchical Bayesian methods, a fundamental challenge arises. This challenge involves determining the signal-to-noise ratio (SNR) of gravitational wave signals across a vast number of events, potentially reaching millions. Such a task, with traditional computational approaches, is both time-intensive and impractical for in-depth analysis. The Python package `gwsnr` addresses this bottleneck by providing an efficient and precise methodology for calculating the SNR of gravitational wave signals.

## Key Features of `gwsnr`

The `gwsnr` package leverages advanced techniques to enhance computational performance significantly:

1. **Innovative Interpolation Method**: Utilizing a novel half-scaling approach, `gwsnr` interpolates the SNR for gravitational waves originating from spin-less binary systems. The interpolation is executed with respect to the mass parameters of the binary system, ensuring accuracy and efficiency.

2. **Inner Product Calculation with Multiprocessing**: In a manner akin to the SNR calculation approach in the `bilby` package, `gwsnr` employs a noise-weighted inner product method. This technique is adept at handling gravitational wave signals from spin-precessing binary systems. Unique to `gwsnr` is the integration of multiprocessing, which facilitates the parallel processing of complex computations, significantly speeding up the SNR calculation process.

3. **Enhanced Performance with Numba**: The package further optimizes performance by utilizing `numba`'s Just-In-Time (njit) compiler. This approach compiles Python code to machine code at runtime, significantly reducing execution time and enhancing computational efficiency.

These advancements position `gwsnr` as a pivotal tool in gravitational wave data analysis, streamlining the process of SNR calculation in large-scale gravitational wave studies.

# Modified `FINDCHIRP` Method: Half Scaling Approach

## Overview
The Half Scaling method in `gwsnr` offers an efficient interpolation approach for determining the SNR of gravitational wave signals from spin-less binary systems.

## Basis
This method draws upon the analytical SNR formulation presented in [FINDCHIRP](https://arxiv.org/abs/gr-qc/0509116), originally intended for inspiral-only waveforms. However, the Half Scaling approach extends its applicability to encompass non-spinning IMR (inspiral-merger-ringdown) waveforms.

## Implementation
1. **Identification and Interpolation of Critical Formula Components**: The most computationally intensive segment of the formula, denoted as 'halfsnr', is interpolated over a mass parameter grid (including mass ratio and total mass). This interpolation utilizes a refined 2D cubic spline technique, akin to `scipy`'s implementation. Pre-calculated spline coefficients are stored for efficient reuse.

2. **SNR Calculation**: For specific gravitational wave parameters, 'halfsnr' is computed using the pre-stored spline coefficients, and the remaining parts of the formula are evaluated analytically. All functions involved are optimized using `numba`'s Just-In-Time (njit) compilation.

3. **Validation**: The interpolation method's efficacy is verified through comparative analysis with `bilby`'s inner product method, demonstrating consistent results.

## Mathematical Formulation

* If SNR of simple an inspiral waveform, $\rho$=$\left( \frac{1 Mpc}{D_{eff}} \right)\mathcal{A}_{1Mpc} \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$

* Then Half scaled SNR, $\rho_{1/2}$=$\sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } \sim \sqrt{ 4\int_0^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df }$

* For general spinless IMR waveform, if SNR=$\rho$, then Half scaled SNR=$\rho_{1/2}$=$\left( \frac{D_{eff}}{1 Mpc} \right)\mathcal{A}_{1Mpc}^{-1} \times \rho$

* Half scaled SNR can be define as a part of the SNR formula which is independent of the binary system parameters except the mass parameters ($M_{tot},q$).

* It depends on the noise curve of the detector (power spectral density) : psd=$S_n(f)$

Taking care of units:

* $\mathcal{M}$ : chirp mass in $M_{\odot}$, $\mathcal{D}$: luminosity distance in Mpc. 

* $F_+,F_{\times}$: Antenna pattern, $\iota$: inclination angle

$$\mathcal{A}_{1Mpc}=\left(\frac{5}{24\pi}\right)^{1/2}\left(\frac{GM_{\odot}/c^2}{1Mpc}\right)\left(\frac{\pi GM_{\odot}}{c^3}\right)^{-1/6}\left(\frac{\mathcal{M}}{M_{\odot}}\right)^{5/6} $$

\begin{equation}
\mathcal{D}_{eff} = \mathcal{D}\left[ F_+^2\left(\frac{1+cos^2\iota}{2}\right)^2 + F_{\times}^2 cos^2\iota \right]^{-1/2} \tag{3}
\end{equation}

* Half scaled SNR : 
$$ \rho_{1/2} = \left( \frac{D_{eff}}{1 Mpc} \right) \mathcal{M}^{-5/6} \times \rho $$
$$ \rho_{1/2} = \left(\frac{5}{24\pi}\right)^{1/2}\left(\frac{GM_{\odot}/c^2}{1Mpc}\right)\left(\frac{\pi GM_{\odot}}{c^3}\right)^{-1/6}\left(\frac{1}{M_{\odot}}\right)^{5/6} \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } $$

* Putting a upper integration limit at $f_{ISCO}$ lower limit to a chosen $f_{min}$

$$ f_{ISCO} = \frac{f c^3}{ G M_{tot}\pi 6^{3/2} } $$
$$ \rho_{1/2} = \left(\frac{5}{24\pi}\right)^{1/2}\left(\frac{GM_{\odot}/c^2}{1Mpc}\right)\left(\frac{\pi GM_{\odot}}{c^3}\right)^{-1/6}\left(\frac{1}{M_{\odot}}\right)^{5/6} \sqrt{ 4\int_{f_{min}}^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df } $$

# Noise-Weighted Inner Product Method

## Scope
This method is adept at calculating the SNR for gravitational wave signals from spin-precessing binary systems or any system characterizable by frequency domain waveforms in `lalsimulation`. With `gwsnr`, performance is enhanced through multiprocessing.

## Comparison with Bilby
The noise-weighted inner product method is similar to the approach used in `bilby`.

### Methodology
1. **Waveform Generation**: Utilizing `lalsimulation`, waveforms are generated efficiently. Despite the c-coded and pre-compiled nature of waveform generation, this step remains the most time-consuming.

2. **Antenna Pattern Calculation**: The antenna patterns ($F_{+ \times }$) relative to the source's location and orientation are computed. All associated functions are accelerated using `numba`'s njit feature.

3. **Noise-Weighted Inner Product Calculation**: Employing a given power spectral density array, the noise-weighted inner product is computed using the waveform and antenna pattern data. This step, too, benefits from `numba` njit optimization for enhanced performance.

These methodologies underscore `gwsnr`'s capability to efficiently handle a broad spectrum of gravitational wave signals, making it a valuable tool in gravitational wave data analysis.

## Mathematical Formulation

* Inner product :

\begin{equation}
\left( a | b \right) = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df \tag{4}
\end{equation}

* optimal SNR : 

\begin{equation}
\begin{split}
\rho^2 &\; = \left< \tilde{h} | \tilde{h} \right> \\ 
&\; = (F_+ h_+ + F_{\times} h_{\times})^* \times (F_+ h_+ + F_{\times} h_{\times}) \\
&\; = F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> + 2 F_+ F_{\times} \left< h_+ | h_{\times} \right> \nonumber
\end{split} \tag{5}
\end{equation}

* if $h_+$ and $h_{\times}$ are orthogonal, SNR:

\begin{equation}
\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> } \tag{6}
\end{equation}

* Given inner product is defined as, $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$

* Then optimal SNR can be written as, $\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> }$

# `gwsnr`: Efficient SNR Calculation for Gravitational Wave Signals

## Summary

The `gwsnr` Python package is a pivotal development in gravitational wave (GW) astrophysics, addressing the computational challenges in analyzing compact binary coalescences (CBC) and hierarchical Bayesian methods. It provides an efficient and accurate solution for calculating the signal-to-noise ratio (SNR) of gravitational wave signals, crucial in processing a vast number of events, potentially in the millions. `gwsnr` is designed to overcome the limitations of traditional computational methods, offering a scalable and robust solution for GW research.

## Statement of Need

In gravitational wave research, particularly for CBC populations, the precise calculation of SNR is essential for statistical analysis and hyperparameter identification in hierarchical models. The computational demands of such tasks, given the vast scale of data, are often beyond the capabilities of existing tools. `gwsnr` fills this critical gap, offering advanced interpolation methods and integrating multiprocessing for inner product calculations. The utilization of `numba`'s Just-In-Time (njit) compilation further enhances its performance, making it an indispensable tool for researchers in the field.

## Equations

### Half Scaling Method (Modified FINDCHIRP Method)
- For a simple inspiral waveform, the optimal SNR, $\rho$, is expressed as: $\rho = \left( \frac{1 Mpc}{D_{eff}} \right)\mathcal{A}_{1Mpc} \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$.
- The Half scaled SNR, $\rho_{1/2}$, is approximated as: $\rho_{1/2} = \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } \approx \sqrt{ 4\int_0^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df }$.
- For a general spinless IMR waveform, the Half scaled SNR, $\rho_{1/2}$, is defined as: $\rho_{1/2} = \left( \frac{D_{eff}}{1 Mpc} \right)\mathcal{A}_{1Mpc}^{-1} \times \rho$.
- The Half scaled SNR is a segment of the SNR formula that is independent of the binary system parameters except for the mass parameters ($M_{tot},q$), and it is interpolated with respect to $M_{tot}$ and $q$.
- It relies on the detector's noise curve or power spectral density (psd): $S_n(f)$.

### Noise-Weighted Inner Product Method
- The inner product is defined as: $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$.
- The optimal SNR is then formulated as: $\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> }$.
- Terms $\left< h_+ | h_+ \right>$ and $\left< h_{\times} | h_{\times} \right>$, which involve waveform generation, are obtained through multiprocessing, enhancing performance and efficiency.

## Acknowledgements

# Summary 

Gravitational waves, ripples in spacetime predicted by Einstein's theory of General Relativity, have revolutionized astrophysics since their first detection in 2015. These waves are emitted by cataclysmic events like the merging of black holes or neutron stars, providing a unique window into the cosmos. A critical aspect of gravitational wave analysis is the Signal-to-Noise Ratio (SNR). SNR quantifies the strength of a gravitational wave signal relative to the background noise in a detector, like LIGO or Virgo. This ratio is pivotal in confirming the detection of gravitational waves and extracting astrophysical information from them. However, specific scenarios in gravitational wave research, particularly in simulations of detectable gravitational wave events and in hierarchical Bayesian analysis where selection effects are considered, demand extensive and efficient computation of SNR. This requirement presents a significant challenge, as conventional computational approaches are typically time-consuming and impractical for such specialized and large-scale analyses.

Addressing this computational challenge, the Python package `gwsnr` offers a groundbreaking solution, significantly streamlining the SNR calculation process. It incorporates several advanced computational techniques to enhance efficiency and accuracy. Firstly, it utilizes an innovative interpolation method, employing a half-scaling approach for accurately interpolating the SNR of gravitational waves from spin-less binary systems, focusing on the mass parameters. Secondly, the package features a noise-weighted inner product method, similar to that in the `bilby` package, but enhanced with multiprocessing capabilities. This integration allows for the parallel processing of complex calculations, thereby expediting the SNR computation. Lastly, `gwsnr` leverages the `numba` Just-In-Time (njit) compiler, which optimizes performance by compiling Python code into machine code at runtime, drastically reducing execution times. Beyond these technical merits, `gwsnr` stands out for its user-friendly features and seamless integration with other related software packages, making it not just a powerful tool but also an accessible one for researchers. These attributes position `gwsnr` as an invaluable asset in gravitational wave data analysis, particularly in simulations of detectable binary mergers, calculation of merger rates, determining gravitational wave lensing rates, and in the analysis of selection effects within hierarchical Bayesian frameworks. The package thus represents a significant step forward in gravitational wave research, enabling more precise and efficient exploration of the universe through gravitational wave observations. Additionally, gwsnr is instrumental in the LeR package for calculating detectable rates of both lensed and unlensed gravitational wave events, showcasing its utility in advanced gravitational wave studies.



# Mathematical Formulation

## Modified FINDCHIRP Method: Half Scaling Approach

The `gwsnr` package incorporates the Half Scaling method, which enhances the SNR calculation for gravitational waves from spin-less binary systems. This approach is rooted in the FINDCHIRP algorithm and focuses on non-spinning IMR waveforms. The Half scaled SNR ($\rho_{1/2}$) is a segment of the SNR formula that is independent of the binary system parameters except for the mass parameters ($M_{tot},q$), and it is interpolated with respect to $M_{tot}$ and $q$.

- **Interpolation Method**: It uses a 2D cubic spline technique for interpolating the 'halfsnr' segment, crucial for efficient computation.
- **Equations**:
  - For a simple inspiral waveform, the optimal SNR, $\rho$, is expressed as: $\rho = F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi) \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$; where,
    - $F$ is a function of the luminosity distance ($D_l$), chirp mass ($\mathcal{M}$), inclination angle ($\iota$), polarization angle ($\psi$), right ascension ($\alpha$), declination ($\delta$), and polarization angle ($\psi$). $f$ is the frequency.
    - $S_n(f)$ is the detector's noise curve or power spectral density (psd).
  - The Half scaled SNR, $\rho_{1/2}$, is approximated as: $\rho_{1/2} = \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } \approx \sqrt{ 4\int_0^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df }$.
  - For a general spinless IMR waveform, the Half scaled SNR, $\rho_{1/2}$, is defined as: $\rho_{1/2} = \left( \frac{D_{eff}}{1 Mpc} \right)\mathcal{A}_{1Mpc}^{-1} \times \rho$, where $\rho$ is the optimal SNR.

## Noise-Weighted Inner Product Method
This method is designed for SNR calculation of any system analyzed using frequency domain waveforms in `lalsimulation` (including spin-precessing binary systems).

- **Methodology**: Involves waveform generation, antenna pattern calculation, and noise-weighted inner product computation.
- **Equations**:
  - The inner product is defined as: $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$.
  - The optimal SNR is then formulated as: $\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> }$.
  - Terms $\left< h_+ | h_+ \right>$ and $\left< h_{\times} | h_{\times} \right>$, which involve waveform generation, are obtained through multiprocessing, enhancing performance and efficiency.

These formulations demonstrate the advanced computational capabilities of `gwsnr` in gravitational wave data analysis.

# Summary 

Gravitational waves, ripples in spacetime predicted by Einstein's theory of General Relativity, have revolutionized astrophysics since their first detection in 2015. These waves are emitted by cataclysmic events like the merging of black holes or neutron stars, providing a unique window into the cosmos. A critical aspect of gravitational wave analysis is the Signal-to-Noise Ratio (SNR). SNR quantifies the strength of a gravitational wave signal relative to the background noise in a detector, like LIGO or Virgo. This ratio is pivotal in confirming the detection of gravitational waves and extracting astrophysical information from them. However, specific scenarios in gravitational wave research, particularly in simulations of detectable gravitational wave events and in hierarchical Bayesian analysis where selection effects are considered, demand extensive and efficient computation of SNR. This requirement presents a significant challenge, as conventional computational approaches are typically time-consuming and impractical for such specialized and large-scale analyses.

# Statement of Need

The `gwsnr` Python package addresses the need for efficient SNR computation in gravitational wave research. It innovatively streamlines SNR calculations, enhancing accuracy and efficiency with several advanced techniques. Firstly, it utilizes an innovative interpolation method, employing a half-scaling approach for accurately interpolating the SNR of gravitational waves from spin-less binary systems, focusing on the mass parameters. Secondly, the package features a noise-weighted inner product method, similar to that in the `bilby` package, but enhanced with multiprocessing capabilities. This integration allows for the parallel processing of complex calculations, thereby expediting the SNR computation. Lastly, `gwsnr` leverages the `numba` Just-In-Time (njit) compiler, which optimizes performance by compiling Python code into machine code at runtime, drastically reducing execution times. Beyond these technical merits, `gwsnr` stands out for its user-friendly features and seamless integration with other related software packages, making it not just a powerful tool but also an accessible one for researchers. These attributes position `gwsnr` as an invaluable asset in gravitational wave data analysis, particularly in simulations of detectable binary mergers, calculation of merger rates, determining gravitational wave lensing rates, and in the analysis of selection effects within hierarchical Bayesian frameworks. The package thus represents a significant step forward in gravitational wave research, enabling more precise and efficient exploration of the universe through gravitational wave observations. Additionally, `gwsnr` is instrumental in the `LeR` package for calculating detectable rates of both lensed and unlensed gravitational wave events, showcasing its utility in advanced gravitational wave studies.

# Mathematical Formulation

#### Modified FINDCHIRP Method: Half Scaling Approach
The `gwsnr` package introduces the Half Scaling method for SNR calculations in spin-less binary systems. This method, rooted in the FINDCHIRP algorithm, focuses on non-spinning IMR waveforms and particularly interpolates the Half scaled SNR ($\rho_{1/2}$) based on mass parameters ($M_{tot},q$).

- **Interpolation Method**: Utilizes a 2D cubic spline technique (njit-ted) for the 'halfsnr' segment.
- **Equations**:
  - For a simple inspiral waveform, the optimal SNR: $\rho = F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi) \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$
  - $F$ is defined as a function of luminosity distance ($D_l$), chirp mass ($\mathcal{M}$), inclination angle ($\iota$), polarization angles ($\psi$), right ascension ($\alpha$), and declination ($\delta$). $f$ is the frequency and $S_n(f)$ is the detector's noise curve or power spectral density (psd).
  - Half scaled SNR: $\rho_{1/2} = \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } \approx \sqrt{ 4\int_0^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df }$
  - For spinless IMR (Inspiral-Merger-Ringdown) waveforms: $\rho_{1/2} = \rho\,/\, F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi)$, where $\rho$ is the optimal SNR.

#### Noise-Weighted Inner Product Method
Designed for SNR calculations in systems characterized by frequency domain waveforms in `lalsimulation`, including spin-precessing binary systems.

- **Methodology**: Combines waveform generation (multi-process), antenna pattern calculation (njit-ted), and noise-weighted inner product computation (njit-ted).
- **Equations**:
  - Inner product: $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$
  - Optimal SNR: $\rho = \sqrt{ F_+^2 \left< \tilde{h}_+ | \tilde{h}_+ \right> + F_{\times}^2 \left< \tilde{h}_{\times} | \tilde{h}_{\times} \right> }$, for orthogonal $h_+$ and $h_{\times}$.
  - $h_{+\times}$ are frequency domain waveform polarizations, and $F_{+\times}$ are antenna patterns. 

These formulations highlight `gwsnr`'s capability to efficiently process diverse gravitational wave signals, enhancing data analysis accuracy and efficiency.

Your draft for the JOSS publication is already well-structured and informative. To refine it for publication, I'll focus on enhancing readability, maintaining formal tone, and ensuring the content is comprehensive and original. Here's the revised version:

# Summary

The landmark detection of gravitational waves in 2015, as predicted by Einstein's theory, marked a new era in astrophysics. Originating from monumental cosmic events such as black hole and neutron star mergers, these waves offer insights into the universe's mysteries. Central to their study is the Signal-to-Noise Ratio (SNR), a measure critical for validating wave detections and interpreting astrophysical data. The SNR computation, especially in large-scale analyses like detectable event simulations and hierarchical Bayesian analysis, poses significant computational challenges, rendering traditional methods inefficient.

# Statement of Need

The `gwsnr` Python package emerges as a vital solution for efficient SNR computation in gravitational wave analysis. It brings innovative computational strategies to enhance precision and speed. Key features include:
- An advanced interpolation method, particularly effective for SNR calculation in spin-less binary systems, focusing on mass parameters.
- A noise-weighted inner product technique, improved with multiprocessing, expediting complex calculations.
- Utilization of the `numba` Just-In-Time compiler for optimal performance, significantly reducing runtime.
`gwsnr` excels in user-friendliness and interoperability with other software, proving crucial in diverse gravitational wave analyses, from binary merger simulations to lensing rate evaluations and hierarchical Bayesian framework applications. It also plays a significant role in the `LeR` package for assessing both lensed and unlensed gravitational wave events.

# Mathematical Formulation

#### Modified FINDCHIRP Method: Half Scaling Approach
The `gwsnr` package introduces an efficient Half Scaling method, enhancing SNR calculations for spin-less binary systems. This method, an extension of the FINDCHIRP algorithm, focuses on non-spinning IMR waveforms, interpolating the Half scaled SNR based on mass parameters.

- **Key Equations**:
  - Optimal SNR for inspiral waveforms: $\rho = F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi) \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$
  - Half scaled SNR: $\rho_{1/2} = \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df } \approx \sqrt{ 4\int_0^{f_{ISCO}} \frac{f^{-7/3}}{S_n(f)}df }$

#### Noise-Weighted Inner Product Method
This method caters to SNR calculations for various gravitational waveforms, including those from spin-precessing systems, processed through `lalsimulation`.

- **Key Equations**:
  - Inner product: $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$
  - Optimal SNR: $\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> }$

`gwsnr` efficiently processes a wide range of gravitational wave signals, significantly enhancing the accuracy and efficiency of data analysis in this pioneering field.

The "Mathematical Formulations" section of your draft, focusing on the modified FINDCHIRP method and the Noise-Weighted Inner Product Method, can be enhanced by explicitly defining the variables used in the equations. Here's a revised version:

#### Modified FINDCHIRP Method: Half Scaling Approach
The `gwsnr` package incorporates the Half Scaling method for SNR calculations in non-spinning IMR waveforms. This method focuses on interpolating the Half scaled SNR ($\rho_{1/2}$), particularly concerning the mass parameters ($M_{tot},q$) of binary systems.

- **Equations**:
  - Optimal SNR for inspiral waveforms, $\rho$: Defined as a function of luminosity distance ($D_l$), chirp mass ($\mathcal{M}$), inclination angle ($\iota$), polarization angles ($\psi$), right ascension ($\alpha$), and declination ($\delta$): $\rho = F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi) \sqrt{ 4\int_0^\infty \frac{f^{-7/3}}{S_n(f)}df }$
  - Half scaled SNR, $\rho_{1/2}$: Approximated for non-spinning IMR waveforms as $\rho_{1/2} = \rho\,/\, F(D_l,\mathcal{M},\iota,\psi,\alpha, \delta, \psi)$, focusing on the effective distance ($D_{eff}$) and amplitude at 1 Mpc ($\mathcal{A}_{1Mpc}$).

#### Noise-Weighted Inner Product Method
This method is designed for calculating SNR of gravitational waveforms from `lalsimulation`, including those from spin-precessing binary systems.

- **Equations**:
  - Inner product ($\left< a | b \right>$): Calculated over a frequency range from $f_{min}$ to $f_{max}$, using the formula $\left< a | b \right> = 4 \int_{f_{min}}^{f_{max}} \frac{\tilde{a}(f)\tilde{b}^*(f)}{S_n(f)} df$, where $\tilde{a}(f)$ and $\tilde{b}(f)$ are frequency-domain representations of gravitational wave signals, and $S_n(f)$ is the power spectral density of the detector's noise.
  - Optimal SNR, $\rho$: Derived from antenna patterns ($F_+$ and $F_{\times}$) and inner products of waveforms ($h_+$ and $h_{\times}$): $\rho = \sqrt{ F_+^2 \left< h_+ | h_+ \right> + F_{\times}^2 \left< h_{\times} | h_{\times} \right> }$

These mathematical formulations underscore the advanced computational capabilities of `gwsnr` in gravitational wave data analysis, offering precision and efficiency in processing diverse gravitational wave signals.